# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

W. Brandner  ->  W. Brandner  |  ['W. Brandner']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
T. Henning  ->  T. Henning  |  ['T. Henning']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
G.-D. Marleau  ->  G.-D. Marleau  |  ['G.-D. Marleau']
J. Sauter  ->  J. Sauter  |  ['J. Sauter']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
T. Suhasaria  ->  T. Suhasaria  |  ['T. Suhasaria']
S. M. Wee  ->  S. M. Wee  |  ['S. M. Wee']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
T. Henning  ->  T. Henning  |  ['T. Henning']


Arxiv has 61 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.11953


/tmp/ipykernel_2874/2822249172.py:52: LatexWarning: 2410.11953 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2410.12202


extracting tarball to tmp_2410.12202...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.12235
extracting tarball to tmp_2410.12235... done.


Found 37 bibliographic references in tmp_2410.12235/EA.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.12235-b31b1b.svg)](https://arxiv.org/abs/2410.12235) | **Infrared Spectra of Solid-State Ethanolamine: Laboratory Data in Support of JWST Observations**  |
|| <mark>T. Suhasaria</mark>, et al. -- incl., <mark>S. M. Wee</mark>, <mark>G. Perotti</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2024-10-17*|
|*Comments*| *8 pages, 5 figures*|
|**Abstract**|            Ethanolamine (NH$_2$CH$_2$CH$_2$OH, EA) has been identified in the gas phase of the ISM within molecular clouds. Although EA has not been directly observed in the molecular ice phase, a solid state formation mechanism has been proposed. However, the current literature lacks an estimation of the infrared band strengths of EA ices. We conducted an experimental investigation of solid EA ice at low temperatures to ascertain its infrared band strengths, phase transition temperature, and multilayer binding energy. The commonly used laser interferometry method was not applied. Infrared band strengths were determined using three distinct methods. The obtained lab spectrum of EA was compared with the publicly available MIRI MRS James Webb Space Telescope observations toward a low mass protostar. The phase transition temperature for EA ice falls within the range of 175 to 185 K. Among the discussed methods, the simple pressure gauge method provides a reasonable estimate of band strength. We derive a band strength value of about $1\times10^{-17}$ cm molecule$^{-1}$ for the NH$_2$ bending mode in the EA molecules. Additionally, temperature-programmed desorption analysis yielded a multilayer desorption energy of 0.61$\pm$0.01 eV. By comparing the laboratory data documented in this study with the JWST spectrum of the low mass protostar IRAS 2A, an upper-limit for the EA ice abundances was derived.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.12202-b31b1b.svg)](https://arxiv.org/abs/2410.12202) | **Simultaneous Eruption and Shrinkage of Pre-existing Flare Loops during a Subsequent Solar Eruption**  |
|| H. Chen, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-10-17*|
|*Comments*| *The paper has been accepted for publication in the ApJ*|
|**Abstract**|            We investigated two consecutive solar eruption events in the solar active region (AR) 12994 at the solar eastern limb on 2022 April 15. We found that the flare loops formed by the first eruption were involved in the second eruption. During the initial stage of the second flare, the middle part of these flare loops (E-loops) erupted outward along with the flux ropes below, while the parts of the flare loops (I-loops1 and I-loops2) on either side of the E-loops first rose and then contracted. Approximately 1 hour after the eruption, the heights of I-loops1 and I-loops2 decreased by 9 Mm and 45 Mm, respectively, compared to before the eruption. Their maximum descent velocities were 30 km/s and 130 km/s, respectively. The differential emission measure (DEM) results indicate that the plasma above I-loops1 and I-loops2 began to be heated about 23 minutes and 44 minutes after the start of the second flare, respectively. Within 20 minutes, the plasma temperature in these regions increased from ~3 MK to 6 MK. We proposed an adiabatic heating mechanism that magnetic energy would be converted into thermal and kinetic energy when the pre-stretched loops contract. Our calculations show that the magnetic energy required to heat the two high-temperature regions are 10^29-10^30 erg, which correspond to a loss of field strength of 2-3 G.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.11953-b31b1b.svg)](https://arxiv.org/abs/2410.11953) | **The cool brown dwarf Gliese 229 B is a close binary**  |
|| J. W. Xuan, et al. -- incl., <mark>W. Brandner</mark>, <mark>G. Chauvin</mark>, <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>L. Kreidberg</mark>, <mark>G.-D. Marleau</mark>, <mark>J. Sauter</mark> |
|*Appeared on*| *2024-10-17*|
|*Comments*| *Published in Nature. The Version of Record of this article is located at this https URL*|
|**Abstract**|            Owing to their similarities with giant exoplanets, brown dwarf companions of stars provide insights into the fundamental processes of planet formation and evolution. From their orbits, several brown dwarf companions are found to be more massive than theoretical predictions given their luminosities and the ages of their host stars (e.g. Brandt et al. 2021, Cheetham et al. 2018, Li et al. 2023). Either the theory is incomplete or these objects are not single entities. For example, they could be two brown dwarfs each with a lower mass and intrinsic luminosity (Brandt et al. 2021, Howe et al. 2024). The most problematic example is Gliese 229 B (Nakajima et al. 1995, Oppenheimer et al. 1995), which is at least 2-6 times less luminous than model predictions given its dynamical mass of $71.4\pm0.6$ Jupiter masses ($M_{\rm Jup}$) (Brandt et al. 2021). We observed Gliese 229 B with the GRAVITY interferometer and, separately, the CRIRES+ spectrograph at the Very Large Telescope. Both sets of observations independently resolve Gliese 229 B into two components, Gliese 229 Ba and Bb, settling the conflict between theory and observations. The two objects have a flux ratio of $0.47\pm0.03$ at a wavelength of 2 $\mu$m and masses of $38.1\pm1.0$ and $34.4\pm1.5$ $M_{\rm Jup}$, respectively. They orbit each other every 12.1 days with a semimajor axis of 0.042 astronomical units (AU). The discovery of Gliese 229 BaBb, each only a few times more massive than the most massive planets, and separated by 16 times the Earth-moon distance, raises new questions about the formation and prevalence of tight binary brown dwarfs around stars.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.12235/./Fig5.png', 'tmp_2410.12235/./Fig1.png', 'tmp_2410.12235/./Fig3.png']
copying  tmp_2410.12235/./Fig5.png to _build/html/
copying  tmp_2410.12235/./Fig1.png to _build/html/
copying  tmp_2410.12235/./Fig3.png to _build/html/
exported in  _build/html/2410.12235.md
    + _build/html/tmp_2410.12235/./Fig5.png
    + _build/html/tmp_2410.12235/./Fig1.png
    + _build/html/tmp_2410.12235/./Fig3.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\textmathbf}[1]{\textbf{\boldmath#1}}$</div>



<div id="title">

# Infrared Spectra of Solid-State Ethanolamine: Laboratory Data in Support of JWST Observations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.12235-b31b1b.svg)](https://arxiv.org/abs/2410.12235)<mark>Appeared on: 2024-10-17</mark> -  _8 pages, 5 figures_

</div>
<div id="authors">

<mark>T. Suhasaria</mark>, et al. -- incl., <mark>S. M. Wee</mark>, <mark>G. Perotti</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Ethanolamine ($NH_2$ $CH_2$ $CH_2$ OH, EA) has been identified in the gas phase of the interstellar medium within molecular clouds. Although EA hasn't been directly observed in the molecular ice phase, a solid-state formation mechanism has been proposed. However, the current literature lacks an estimation of the infrared band strengths of EA ices, which are crucial data for quantifying potential astronomical observations and laboratory findings related to their formation or destruction via energetic processing. We conducted an experimental investigation of solid EA ice at low temperatures to ascertain its infrared band strengths, phase transition temperature, and multilayer binding energy. Since the refractive index and the density of EA ice are unknown, the commonly used laser interferometry method was not applied. Infrared band strengths were determined using three distinct methods. Besides the evaluation of band strengths of EA, we also tested the advantages and disadvantages of different approaches used for this purpose. The obtained lab spectrum of EA was compared with the publicly available MIRI MRS James Webb Space Telescope observations toward a low mass protostar. We used a combination of Fourier-transform transmission infrared spectroscopy and quadrupole mass spectrometry. The phase transition temperature for EA ice falls within the range of 175 to 185 K. Among the discussed methods, the simple pressure gauge method provides a reasonable estimate of band strength. We derive a band strength value of about $1\times10^{-17}$ cm molecule $^{-1}$ for the $NH_2$ bending mode in the EA molecules. Additionally, temperature-programmed desorption analysis yielded a multilayer desorption energy of 0.61 $\pm$ 0.01 eV. By comparing the laboratory data documented in this study with the JWST spectrum of the low mass protostar IRAS 2A, an upper-limit for the EA ice abundances was derived. This study addresses the lack of quantitative infrared measurements of EA at low temperatures, crucial for understanding its astronomical and laboratory presence and formation routes. Our approach presents a simple, yet effective method for determining the infrared band strengths of molecules with a reasonable level of accuracy.

</div>

<div id="div_fig1">

<img src="tmp_2410.12235/./Fig5.png" alt="Fig4" width="100%"/>

**Figure 4. -** Comparison of the JWST/MIRI MRS spectra of IRAS 2A (blue curve) and IRAS 2A with water:silicate features subtracted (red curve) to the laboratory-measured EA ice profile at 10 K. For clearer comparison, all spectra are vertically shifted, with the laboratory spectrum intensity is multiplied by 20, and the EA band at 1607 cm$^{-1}$ is indicated by dashed line. The inset provides a closer examination of the water:silicate subtracted IRAS 2A spectrum alongside the fitted EA profile for upper limit estimation. (*fig5*)

</div>
<div id="div_fig2">

<img src="tmp_2410.12235/./Fig1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Experimental and theoretical mid-IR spectrum of EA. The upper one shows about 200 monolayers (ML) EA condensed on KBr at 10 K. The infrared bands marked with asterisks are due to a mixture of different vibrational modes. For the latter, calculations were performed at MP2/Aug-cc-pVTZ level. (*fig1*)

</div>
<div id="div_fig3">

<img src="tmp_2410.12235/./Fig3.png" alt="Fig2" width="100%"/>

**Figure 2. -** Mid-infrared spectra of pure EA and reference molecules $NH_3$ and $CH_3$OH at 10 K. The spectrum of $H_2$O (common interstellar ice component) at 15 K is taken from the Leiden Ice Database for Astrochemistry (LIDA). Bands selected for band strength estimation are indicated by the shaded areas. (*fig3*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.12235"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

221  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
